## Importing Libraries

In [ ]:
%reset -f
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# for reproducability
np.random.seed(seed = 42)

## Preparing the Data

In [ ]:
# read data
movies = pd.read_csv("data/movies.csv") # we'll use this dataframe also later to retrieve title and genres
ratings = pd.read_csv("data/ratings.csv")

print("Columns of movies dataset: ",movies.columns)
print("Columns of ratings dataset: ",ratings.columns)

print("\nNumber of movies in dataset: ", len(movies.movieId.unique()))
print("Number of users in dataset: ", len(ratings.userId.unique()))

# we merge two dataframe to have all the movies including the ones which are not voted
df = pd.merge(ratings, movies, on = "movieId", how="right")

# to obtain the desired matrix form we use pandas.pivot_table function
# users are on the rows, movies are on the columns

C = df.pivot_table(index = "userId", columns = "movieId", values = "rating", dropna=False)

# just to have correct datatype (it is changed because of NA values)
C.index = C.index.astype(int)

C

In [ ]:
# each user has at least 20, at max 2698 ratings among the 9742 movies
print("The least voted user has voted {} times".format(C.notnull().sum(axis=1).min()))
print("The most voted user has voted {} times".format(C.notnull().sum(axis=1).max()))

# some movies never rated, at most they rated 329 times by different users
print("The least rated movie has voted {} times".format(C.notnull().sum(axis=0).min()))
print("The most rated movie has voted {} times".format(C.notnull().sum(axis=0).max()))

# minimum and maximum rating
print("Minimum rating:",C.min(axis=0).min())
print("Maximum rating:",C.max(axis=0).max())

In [ ]:
# all unobserved values will be filled with 0 for the sake of cost function
C.fillna(0,inplace=True)
C

## Weighted Alternating Least Squares

In [ ]:
num_user, num_item = C.shape
num_factor = 50 # I decided number of features, this number will be decided by using CV later

$$
\sum_{i,j \in Obs}{w_{i,j}(C_{i,j} - U_{i,j} V_{i,j}^T)^2} + \lambda {(\lVert U \rVert^2 + \lVert V \rVert^2)}
$$

In [ ]:
def cost_function(C, U, V, reg, w):
    C_prime = np.dot(U, V.T)
    return w * np.sum((C.values - C_prime) ** 2) + reg * (np.linalg.norm(U) ** 2 + np.linalg.norm(V) ** 2)

To optimize the cost function, we start by setting one of the vectors as constant. For this example we can use item vector $V$. Then we can take the derivate of the loss function with respect to other vector. For the sake of simplicity $i,j$ won't be showed in the notation

$$
\frac{\partial L}{\partial U} = -2 \sum{w(C - U V^T) V} + 2 \lambda U = 0 \\
= - w (C - U V^T) V + \lambda U = 0 \\
= U(w V^T V + \lambda I) = w C V \\
= U = w C V (w V^T V + \lambda I)^{-1}
$$

Similarly, we can calculate the $V$ by following same procedure.

$$
\frac{\partial L}{\partial V} = V = w C U (w U^T U + \lambda I)^{-1}
$$

In [ ]:
# this is the analytical solution, please note that instead of lambda 
# we used reg as a parameter since lambda exist in python as keyword

def recompute_factors(ratings, fixed_vecs, solve_vecs, num_factor, reg , w):
    
    A = w * fixed_vecs.T.dot(fixed_vecs) + np.eye(num_factor) * reg
    b = w * ratings.dot(fixed_vecs)
    A_inv = np.linalg.inv(A)
    solve_vecs = b.dot(A_inv)
    return solve_vecs

In [ ]:
def WALS(C, num_factor, num_user, num_item, reg, num_iter, verbose = False):
    
    cost_list = []                                  # cost_list is for storing cost for each iteration
    # randomly initialize the matrices U and V
    U = np.random.rand(num_user, num_factor)
    V = np.random.rand(num_item, num_factor)
    
    if verbose == True:
        print("Iteration \t\t Cost")
        
    for i in range(1,num_iter+1): # range(1,num_iter +1) to start the iteration from 1 not 0
        
        # fix U and find V , reg and w are chosen by arbitrary (w has no effect on the observations)
        V = recompute_factors(C.T.values, U, V, num_factor, reg , 1)
        # fix V and find U
        U = recompute_factors(C.values, V, U, num_factor, reg, 1)
        
        cost = cost_function(C, U, V, reg, 1)
        
        if verbose == True:
            print(i,"\t\t",cost)
            
        cost_list.append(cost)
    
    # return dataframe which is similar to C
    C_prime = pd.DataFrame(np.dot(U, V.T), columns = C.columns, index = C.index)
        
    return C_prime, cost_list

In [ ]:
%%time
C_prime, cost_list = WALS(C, num_factor, num_user, num_item, 0.5, 100, verbose = True)

In [ ]:
plt.figure(figsize = (12,8))
plt.title("Change in Cost with Iterations")
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.plot(cost_list);

## Recommendation

In [ ]:
def recommend(C, C_prime, user_id, num_recommendation):
    mask = np.where(C == 0)                        # filtering the movies which is not watched
    user = user_id - 1                        # in numpy, indexing starts with 0, in dataframe it is 1

    user_column = mask[1][mask[0] == user]    # to filter movies (were in columns) in C_prime
    
    scores = C_prime.iloc[user,user_column]
    scores_reduced = scores.sort_values(ascending = False)[:num_recommendation]   # sort the scores descending order
    
    df = pd.DataFrame(scores_reduced)
    df.reset_index(inplace=True)                                 # to get movieId as column
    df.columns = ["movieId", "Score"]
    df = pd.merge(df,movies,on="movieId")                        # to get also title and genres
    
    print("Movie Recommendations for user {}".format(user_id))

    return df            

In [ ]:
recommend(C, C_prime, 455, num_recommendation=10)

## Deciding Parameters with CV

### Latent Factor

In [ ]:
# to use it in cross validation
def calculate_test_error(array1, array2):
    return np.sum(np.square(array1 - array2))

In [ ]:
def param_CV(C, param_list, fold, latent_factor = True):
    # mask the observations
    mask = np.nonzero(C.values)
    idx = len(C.values[mask]) // fold  # will be used for indexing mask, in other words test size
    
    param_error = {}                # store the results for each param in dictionary
    for param in param_list:
        print("Evaluating parameter {}".format(param))

        cost = []
        for i in range(fold): 

            C_train = C.copy()              
            test_mask_row = mask[0][i*idx : (i+1)*idx]   # preparing test set
            test_mask_column = mask[1][i*idx : (i+1)*idx]

            # keep those values in a seperated array
            test_array = C_train.values[test_mask_row, test_mask_column]

            # set those values to 0, like we didn't observe them at all
            C_train.iloc[test_mask_row, test_mask_column] = 0.0
            
            # this if statement is just for using the function for latent_factor and reg
            if latent_factor == True:
                # now calculate the C_prime, 30 iteration is enough to converge and save time from execution
                C_prime, cost_list = WALS(C_train, param, C.shape[0], C.shape[1], 0.5, 30)
            else:
                C_prime, cost_list = WALS(C_train, 50, C.shape[0], C.shape[1], param, 30)

            # get the approximated results for test dataset
            predicted_array = C_prime.values[test_mask_row, test_mask_column]

            # calculate the test error and store it
            test_error = calculate_test_error(test_array, predicted_array)
            cost.append(test_error)
              
        param_error[param] = cost
        
    return param_error

In [ ]:
%%time
# this part may take time since there is fold * len(param_list) operation

num_latent_list = np.linspace(10,100,10, dtype=np.int)

latent_factor_CV_error = param_CV(C, num_latent_list, 5)
# construct a dataframe from errors
latent_factors_error_df = pd.DataFrame(latent_factor_CV_error, dtype=np.float)

# first take the mean of dataframe columns-wise and sort 
# it according to index to find the minimum among the columns

idx = np.argsort(latent_factors_error_df.mean(axis=0).values)
latent_factors_ordered = latent_factors_error_df.columns[idx]
print("Best number of latent factor is {}".format(latent_factors_ordered[0]))

### Regularizaiton $\lambda$

In [ ]:
%%time

# same process for regularization parameter to get the one which has the smallest error mean

reg_list = np.linspace(0.1,1,10, dtype=np.float)

reg_CV_error = param_CV(C, reg_list, 5 ,latent_factor = False)
reg_error_df = pd.DataFrame(reg_CV_error, dtype=np.float)

idx = np.argsort(reg_error_df.mean(axis=0).values)
reg_ordered = reg_error_df.columns[idx]
print("Best number of regularization parameter is {}".format(reg_ordered[0]))

## Cold Start Problem

In [ ]:
def recommend_with_cold_start(C, C_prime, user_id, num_recommendation):
    # check that user has no rating before, if yes return the most popular movies
    # simply by taking the mean of the movies
    
    user_votes = C.loc[user_id, :].unique()
    
    if len(user_votes) == 1 and user_votes[0] == 0:
        popular_movies = C.mean(axis=0).sort_values()[::-1][:num_recommendation]
        df = pd.DataFrame(popular_movies)
        df.reset_index(inplace=True)                                 # to get movieId as column
        df.columns = ["movieId", "Score"]
        df = pd.merge(df,movies,on="movieId")                        # to get also title and genres
        print("Movie Recommendations for user {}".format(user_id))
        return df
    else:
        # call normal recommend function
        recommend(C, C_prime, user_id, num_recommendation)
    

In [ ]:
# add new user to system which has no ratings
C.loc[611,:] = np.zeros(len(C.columns))

# constructing C_prime with new C by using parameters that we found, actually make no difference
# for new users since we recommend them the most popular ones
C_prime, cost_list = WALS(C, 40, C.shape[0], C.shape[1], 0.3, 100)

recommend_with_cold_start(C, C_prime, 611,10)

## References

http://files.grouplens.org/datasets/movielens/ml-latest.zip

https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf

https://developers.google.com/machine-learning/recommendation/collaborative/matrix

http://ethen8181.github.io/machine-learning/recsys/1_ALSWR.html

https://www.ethanrosenthal.com/2016/01/09/explicit-matrix-factorization-sgd-als/